In [1]:
from functions import functions as fc
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Grab nationalities with more reviews (TOP10)
q = "SELECT reviewer_country, COUNT(*) as totalReviews FROM reviews GROUP BY reviewer_country ORDER BY totalReviews DESC LIMIT 10"
nationalities_res = fc.db.select(q, all=True)
nationalities = [str(n['reviewer_country']) for n in nationalities_res]
b_q = "SELECT idbusiness from business where category = 1 order by review_count"
business_res = fc.db.select(b_q, all=True)
business = [str(b['idbusiness']) for b in business_res]

# Get the reviews for this business of the top 10 nationalities
r_q = "SELECT r.idBusiness, reviewer_score, reviewer_score_cal, reviewer_country, rating, rating_cal, country FROM reviews r INNER JOIN business b ON r.idbusiness = b.idbusiness WHERE r.idbusiness IN (" + (',').join(business) + ") AND reviewer_country IN ("+ (',').join(nationalities) +")"
reviews = fc.db.select(r_q, all=True)
review_df = pd.DataFrame(reviews)
bins = [0, 0.65, 0.85, 1]
labels = [1, 2, 3]
review_df['b_review'] = pd.cut(review_df['rating_cal'], bins, labels=labels)
review_df['u_review'] = pd.cut(review_df['reviewer_score_cal'], bins, labels=labels)
model_data = review_df.drop(['idBusiness','reviewer_score','reviewer_score_cal','rating_cal','rating'], axis =1)
model_data_no_bin = review_df.drop(['idBusiness','reviewer_score','rating','b_review','u_review'], axis =1)

In [19]:
final_data = model_data[model_data['reviewer_country'] == 235].drop('reviewer_country', axis=1)
final_data

,country,b_review,u_review
2,157,3,2
3,157,3,2
4,157,3,3
5,157,3,3
6,157,3,2
...,...,...,...
366453,15,3,3
366457,15,3,3
366458,15,3,3
366459,15,3,2


In [20]:
pred = 'u_review'
X = final_data.drop(columns=[pred])
y = final_data[pred].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(194928, 2) (48733, 2) (194928,) (48733,)


In [41]:
print(X_train)

       country b_review
177754     235        3
338347     235        3
146574     235        3
152531     235        3
98634      235        3
...        ...      ...
243137     235        3
182373     235        3
58153      235        3
359364     235        3
307122     157        3

[194928 rows x 2 columns]


In [36]:
# KNN

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set:', knn.score(X_train, y_train))
print('Accuracy of K-NN classifier on test set:', knn.score(X_test, y_test))

Accuracy of K-NN classifier on training set: 0.877867725519166
Accuracy of K-NN classifier on test set: 0.8812098577965649


In [37]:
# Logistic Regression

logreg = LogisticRegression().fit(X_train, y_train)
print('Accuracy of Logistic regression classifier on training set:', logreg.score(X_train, y_train))
print('Accuracy of Logistic regression classifier on test set:' , logreg.score(X_test, y_test))

Accuracy of Logistic regression classifier on training set: 0.8778369449232537
Accuracy of Logistic regression classifier on test set: 0.8813534976299428


In [38]:
# Decision Tree

clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set:', clf.score(X_train, y_train))
print('Accuracy of Decision Tree classifier on test set:', clf.score(X_test, y_test))

Accuracy of Decision Tree classifier on training set: 0.8778985061150784
Accuracy of Decision Tree classifier on test set: 0.8812508977489586


In [39]:
# Random Forest Classifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print('Accuracy of Random Forest classifier on training set:', rf.score(X_train, y_train))
print('Accuracy of Random Forest classifier on test set:', rf.score(X_test, y_test))

Accuracy of Random Forest classifier on training set: 0.8778985061150784
Accuracy of Random Forest classifier on test set: 0.8812508977489586


In [26]:
knn_pred = knn.predict(X_test)

In [27]:
logreg_pred = logreg.predict(X_test)

In [28]:
clf_pred = clf.predict(X_test)

In [29]:
rf_pred = rf.predict(X_test)

In [40]:
model_data

,reviewer_country,country,b_review,u_review
0,108,157,3,3
1,14,157,3,3
2,235,157,3,2
3,235,157,3,2
4,235,157,3,3
...,...,...,...,...
366457,235,15,3,3
366458,235,15,3,3
366459,235,15,3,2
366460,235,15,3,2


In [15]:
print(y_test)
print(knn_pred)
print(logreg_pred)
print(clf_pred)
print(rf_pred)

[2, 3, 3, 3, 3, ..., 3, 3, 3, 2, 1]
Length: 6990
Categories (3, int64): [1 < 2 < 3]
[3 3 3 ... 3 3 3]
[3 3 3 ... 3 3 3]
[3 3 3 ... 3 3 3]
[3 3 3 ... 3 3 3]


In [34]:
test_input = pd.DataFrame([157,1]).T
print(knn.predict(test_input))
# test_input

[2]
